In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install openai
!pip install tiktoken

In [ ]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [ ]:
!unzip -q pdfs.zip -d pdfs

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
data = loader.load()

In [ ]:
data

[Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-24T00:37:53+00:00', 'author': '', 'keywords': '', 'moddate': '2024-10-24T00:37:53+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs/pdfs/yolo11.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='YOLO V11: A N OVERVIEW OF THE KEY ARCHITECTURAL\nENHANCEMENTS\nRahima Khanam* and Muhammad Hussain\nDepartment of Computer Science, Huddersfield University, Queensgate, Huddersfield HD1 3DH, UK;\n*Correspondence: rahima.khanam@hud.ac.uk;\nOctober 24, 2024\nABSTRACT\nThis study presents an architectural analysis of YOLOv11, the latest iteration in the YOLO (You Only\nLook Once) series of object detection models. We examine the models architectural innovations,\nincluding the introduction of the C3k2 (Cross Stage Partial with kernel size 2) b

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
len(text_chunks)

81

In [ ]:
text_chunks[1]

Document(metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-10-24T00:37:53+00:00', 'author': '', 'keywords': '', 'moddate': '2024-10-24T00:37:53+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'pdfs/pdfs/yolo11.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1'}, page_content='including the introduction of the C3k2 (Cross Stage Partial with kernel size 2) block, SPPF (Spatial\nPyramid Pooling - Fast), and C2PSA (Convolutional block with Parallel Spatial Attention) com-\nponents, which contribute in improving the models performance in several ways such as enhanced\nfeature extraction. The paper explores YOLOv11’s expanded capabilities across various computer\nvision tasks, including object detection, instance segmentation, pose estimation, and oriented object')

Download Embeddings

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = ""

In [ ]:
embeddings = OpenAIEmbeddings()

<ipython-input-14-73ad2f8e367a>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [ ]:
result = embeddings.embed_query("Hello")

In [ ]:
len(result)

1536

Intialize the PineCone

In [ ]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV','us-east-1')

In [ ]:
texts = [doc.page_content for doc in text_chunks]
metadatas = [doc.metadata for doc in text_chunks]
vectors = embeddings.embed_documents(texts)

In [ ]:
client = pinecone.Pinecone(api_key=PINECONE_API_KEY)
index = client.Index(index_name)

items = [
    {"id": f"doc-{i}", "values": vector, "metadata": metadatas[i]}
    for i, vector in enumerate(vectors)
]
index.upsert(vectors=items)

print("Documents successfully indexed.")

Documents successfully indexed.


In [ ]:
pip install -U langchain-pinecone


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00


In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_core.vectorstores import VectorStoreRetriever

In [ ]:
vectorstore = PineconeVectorStore(
    index=index,
    embedding=embeddings,
    text_key="text",
)

retriever = vectorstore.as_retriever()

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = ""

In [ ]:
from langchain_pinecone import PineconeVectorStore

texts = [t.page_content for t in text_chunks]

# Create and index embeddings (this performs upsert automatically)
docsearch = PineconeVectorStore.from_texts(
    texts=texts,
    embedding=embeddings,
    index_name="test",  # your existing index name
)


In [ ]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)

In [ ]:
docsearch

In [ ]:
query = "YOLOv11 outperforms which models"

In [ ]:
docs = docsearch.similarity_search(query, k=3)
docs

[Document(id='ac145641-ae9d-4e42-8946-ef15fa93e08a', metadata={}, page_content='YOLOv11 model demonstrates significant advancements in both inference speed and accuracy compared to its\npredecessors. In the benchmark analysis, YOLOv11 was compared against several of its predecessors including variants\nsuch as YOLOv5 [24] through to the more recent variants such as YOLOv10. As presented in Figure 2, YOLOv11\nconsistently outperforms these models, achieving superior mAP on the COCO dataset while maintaining a faster\ninference rate [25].'),
 Document(id='a568a53a-d0ee-4d7e-b973-b795fceeaebb', metadata={}, page_content='YOLO11’s overall performance improvements, particularly in challenging visual environments.\n5. Performance Benchmarks: Comparative analyses reveal YOLO11’s superior performance, particularly in its\nsmaller variants. The nano model, despite a slight increase in parameters, demonstrates enhanced inference\nspeed and frames per second (FPS) compared to its predecessor. Thi

In [ ]:
llm = OpenAI()

<ipython-input-45-4ab60154b5e1>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())


In [ ]:
qa.run(query)

<ipython-input-47-fbc3ad05536a>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa.run(query)


' YOLOv5, YOLOv10'

In [ ]:
query = "Pranav Balaji R S experience and certifications"

In [ ]:
qa.run(query)

' Pranav Balaji R S has experience with frameworks such as TensorFlow, PyTorch, SciKit-Learn, Flask, keras, and Hugging Face, as well as tools and platforms such as Git, GitHub, Jupyter, and VS Code. He also has certifications in TensorFlow, AI and ML Externship, and Google Developers. Additionally, he has experience as a Project Author at the Tech Researchers Club at VIT Chennai.'